In [99]:
import pandas as pd
import numpy as np
import sqlite3
import datetime
import sklearn
import sqlalchemy as sa
from tqdm import tqdm_notebook
import itertools

In [100]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc

import operator
import gc

In [101]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [102]:
%matplotlib inline

In [103]:
%%time
train_raw = pd.read_feather('../../data/prepaired_dataset/train_v2.fth')

CPU times: user 811 ms, sys: 2.19 s, total: 3.01 s
Wall time: 5.76 s


In [104]:
train_raw = train_raw.loc[train_raw.label != -1].copy()

In [105]:
rng = np.random.RandomState(seed=1)

In [106]:
uid_train, uid_val = train_test_split(sorted(train_raw['user_id'].unique()), test_size = 0.2, random_state = rng)

In [111]:
set(uid_train) & set(uid_val)

set()

In [107]:
drop_cols = ['event_id', 'user_id', 'event_time', 'short_date']

In [113]:
train_raw['short_date'].min(), train_raw['short_date'].max()

(20171029, 20171128)

In [17]:
%%time
t = train_raw.loc[train_raw['user_id'].isin(uid_train)].drop(drop_cols, axis = 1)
dtrain = xgb.DMatrix(t.drop('label', axis = 1), label=t['label'], missing=np.NAN)
t = train_raw.loc[train_raw['user_id'].isin(uid_val)].drop(drop_cols, axis = 1)
dval = xgb.DMatrix(t.drop('label', axis = 1), label=t['label'], missing=np.NAN)

CPU times: user 5.27 s, sys: 6.24 s, total: 11.5 s
Wall time: 11.5 s


In [18]:
del t, train_raw
gc.collect()

311

In [49]:
params={
    'max_depth': [3, 5], 
    'subsample': [0.4, 0.6], 
    'colsample_bytree': [0.5, 0.7], 
    'n_estimators': [150, 250],
    'reg_alpha': [0.01, 0.03] 
}

In [73]:
cv_params_list = []
def cv_get(nxt, already = []):
    global cv_params_list
    if len(nxt) == 0:
        return None
    if len(nxt) == 1:
        for k, v in nxt:
            for vv in v:
                cv_params_list.append(already + [(k, vv)])
    else:
        k, v = nxt.pop()
        for vv in v:
            already_cp = already.copy()
            already_cp.append((k, vv))
            cv_get(nxt.copy(), already_cp)
            
cv_get([(k, v) for k,v in sorted(params.items(), key = lambda x: x[0])])
len(cv_params_list)

32

In [81]:
print('XGBoost with grid search and early stopping')
# 
evallist = [(dtrain, 'train'), (dval, 'eval')]
num_round = 250
models = []

for chunk_params in tqdm_notebook([{k:v for k, v in x} for x in cv_params_list]):
    i_params = chunk_params.copy()
    i_params['n_jobs'] = 16
    i_params['objective'] = 'binary:logistic'
    bst = xgb.train(i_params, dtrain, num_round, evallist, verbose_eval=100, early_stopping_rounds = 20)
    models.append((chunk_params, bst))

XGBoost with grid search and early stopping


HBox(children=(IntProgress(value=0, max=32), HTML(value='')))

[0]	train-error:0.003682	eval-error:0.003629
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 20 rounds.
[100]	train-error:0.003036	eval-error:0.003088
Stopping. Best iteration:
[97]	train-error:0.003045	eval-error:0.003068

[0]	train-error:0.003682	eval-error:0.003629
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 20 rounds.
[100]	train-error:0.003021	eval-error:0.003094
Stopping. Best iteration:
[87]	train-error:0.003049	eval-error:0.003066

[0]	train-error:0.003682	eval-error:0.003629
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 20 rounds.
Stopping. Best iteration:
[64]	train-error:0.002855	eval-error:0.003021

[0]	train-error:0.003677	eval-error:0.00364
Multiple eval metrics have been passed: 'eval-error' will be used for ear

Will train until eval-error hasn't improved in 20 rounds.
[100]	train-error:0.00305	eval-error:0.003069
Stopping. Best iteration:
[89]	train-error:0.003077	eval-error:0.003038

[0]	train-error:0.003682	eval-error:0.003629
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 20 rounds.
[100]	train-error:0.003035	eval-error:0.00312
Stopping. Best iteration:
[90]	train-error:0.003058	eval-error:0.00311

[0]	train-error:0.003625	eval-error:0.003613
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 20 rounds.
Stopping. Best iteration:
[58]	train-error:0.002861	eval-error:0.002994

[0]	train-error:0.003675	eval-error:0.003627
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 20 rounds.
Stopping. Best iteration:
[61]	train-error:0.002846	eval-error

In [86]:
from sklearn.metrics import roc_auc_score

In [98]:
for n, (_, m) in enumerate(models):
    print(n, roc_auc_score(dval.get_label(), m.predict(dval)))

0 0.9745258151694256
1 0.9743437221890158
2 0.9799422667593966
3 0.9820631318228367
4 0.9745258151694256
5 0.9743437221890158
6 0.9799422667593966
7 0.9820631318228367
8 0.9743927735440956
9 0.9743864150544304
10 0.9801170827755024
11 0.978450818326852
12 0.9743927735440956
13 0.9743864150544304
14 0.9801170827755024
15 0.978450818326852
16 0.9744473924397042
17 0.9761290488899776
18 0.9806959438795118
19 0.9816410025576936
20 0.9744473924397042
21 0.9761290488899776
22 0.9806959438795118
23 0.9816410025576936
24 0.9744783357322356
25 0.9744713496248483
26 0.9801490681877287
27 0.980893441899248
28 0.9744783357322356
29 0.9744713496248483
30 0.9801490681877287
31 0.980893441899248


In [97]:
bst.get_score()

{'age_diff': 194,
 'amnt2chnls': 162,
 'amount': 163,
 'cat_new_ip': 47,
 'cat_new_prov': 37,
 'cdf_s_127': 43,
 'cdf_s_129': 43,
 'cdf_s_130': 66,
 'cdf_s_133': 31,
 'cdf_s_134': 47,
 'cdf_s_135': 26,
 'channel_indicator_desc_is_MOBILEAPI': 8,
 'channel_indicator_desc_is_WEBAPI': 9,
 'client_age': 254,
 'client_region': 85,
 'cumulative_sum_total': 144,
 'data_i_120': 67,
 'data_i_154': 120,
 'event_day': 56,
 'event_day_is_weekend': 2,
 'event_hour': 78,
 'event_hour_night': 6,
 'ip_isp': 117,
 'know_recip_card_age': 8,
 'know_recip_power': 75,
 'krp_pow2': 31,
 'log_amount': 53,
 'one_region': 25,
 'op_type': 36,
 'recip_card_age': 18,
 'relative': 12,
 'transfer_age_diff': 57,
 'transfer_for_relative': 5,
 'transfer_know_recip_squared': 15,
 'user_id_CRM': 1,
 'user_id_MBK': 8,
 'user_id_digit_only': 7}